In [4]:
import os
import re
import photutils
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
from astropy.modeling import models, fitting
import matplotlib.animation as animation
from photutils.aperture import CircularAperture, aperture_photometry
from photutils.background import Background2D, MedianBackground

data_folder = '/Volumes/Extreme SSD/COSMOS data'

In [9]:
def shift_values(arr, del_x, del_y):
    '''Shift values in an array by a specified discrete displacement.
    
    Parameters
    ----------
    arr : array-like
        The intensity grid.
    del_x : int
        The x displacement, in subpixels.
    del_y : int
        The y displacement, in subpixels.
        
    Returns
    -------
    new_arr : array-like
        The shifted array.
    '''
    n, m = arr.shape
    new_arr = np.zeros_like(arr)
    # print(abs(del_x) > m, abs(del_y) > n)
    new_arr[max(del_y, 0):m+min(del_y, 0), max(del_x, 0):n+min(del_x, 0)] = \
        arr[-min(del_y, 0):m-max(del_y, 0), -min(del_x, 0):n-max(del_x, 0)]
    return new_arr

In [3]:
source_folder = os.path.join(data_folder, '11-01-2024/07 Pleiades Fast')
source_files = [file for file in os.listdir(source_folder) if not file.startswith('.')]

# Function to extract time from filename
def extract_time(filename):
    match = re.search(r'(\d{2}_\d{2}_\d{2})\.fits$', filename)
    return match.group(1) if match else None

# Sort the files by extracted time
source_files_sorted = sorted(source_files, key=extract_time)
source_stack_0 = fits.open(os.path.join(source_folder, source_files_sorted[0]))[0].data

dark_folder = os.path.join(data_folder, '11-01-2024/03 HS_HG_RS Darks')
dark_stack = fits.open(os.path.join(dark_folder, 'HSHG_RS_dark_40ms_-25C 2024 November 02 02_29_36.fits'))[0].data
flat_folder = os.path.join(data_folder, '11-01-2024/03 HS_HG_RS Dome Flats R Filter')
flat_stack = fits.open(os.path.join(flat_folder, 'Dome_Flats_R_Filter_HS_HG_RS_1s_-25C 2024 November 02 02_43_23.fits'))[0].data

In [ ]:
dark_image = np.median(dark_stack, axis=0)
flat_image = np.median(flat_stack, axis=0)
# Normalize the flat image
flat_image /= np.median(flat_image)
plt.imshow(flat_image, vmin=0.9, vmax=1.1)
plt.colorbar()
plt.show()

In [ ]:
%matplotlib qt
subarray_size = 200
image_cube = np.zeros((len(source_stack_0) * len(source_files_sorted), subarray_size, subarray_size))
start_x, start_y = 1330, 1130
shift_x, shift_y = 0, 0

for i, file in enumerate(source_files_sorted[:5]):
    print('Starting File %s' % file)
    start_x -= shift_x
    start_y -= shift_y
    # Clear previous stack_data from memory
    stack_data = fits.open(os.path.join(source_folder, file))[0].data
    subarray_dark = dark_image[start_y - subarray_size // 2:start_y + subarray_size // 2,
                               start_x - subarray_size // 2:start_x + subarray_size // 2]
    subarray_flat = flat_image[start_y - subarray_size // 2:start_y + subarray_size // 2,
                               start_x - subarray_size // 2:start_x + subarray_size // 2]
    for j, frame in enumerate(stack_data):
        subarray_data = frame[start_y - subarray_size // 2:start_y + subarray_size // 2,
                              start_x - subarray_size // 2:start_x + subarray_size // 2]
        subarray_data = (subarray_data - subarray_dark) / subarray_flat
        max_index = np.argmax(subarray_data)
        max_coords = np.unravel_index(max_index, subarray_data.shape)
        print(start_x, start_y, shift_x, shift_y)
        
        y, x = np.mgrid[:subarray_data.shape[0], :subarray_data.shape[1]]
        initial_guess = models.Gaussian2D(amplitude=subarray_data.max(), x_mean=max_coords[1], y_mean=max_coords[0], x_stddev=3, y_stddev=3)
        fitter = fitting.LevMarLSQFitter()
        fitted_gaussian = fitter(initial_guess, x, y, subarray_data)
        shift_x = np.rint(subarray_size / 2 - fitted_gaussian.x_mean.value).astype(int)
        shift_y = np.rint(subarray_size / 2 - fitted_gaussian.y_mean.value).astype(int)
        shifted_frame = shift_values(subarray_data, shift_x, shift_y)
        image_cube[i * len(source_stack_0) + j] = shifted_frame
    del stack_data

In [ ]:
plt.imshow(image_cube.sum(axis=0))
plt.show()

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(image_cube[0], cmap='gray')

def update(frame):
    im.set_array(image_cube[frame])
    return [im]

ani = animation.FuncAnimation(fig, update, frames=len(image_cube), interval=100, blit=True)
plt.show()

In [8]:
positions = [(subarray_size // 2, subarray_size // 2)]  # Center of the subarray

# Define the aperture
aperture = CircularAperture(positions, r=10)
# Create mask to calculate background
source_mask = aperture.to_mask(method='exact')[0].to_image((subarray_size, subarray_size))
source_mask = source_mask.astype(bool)

# Perform aperture photometry on each frame in the image cube
light_curve = np.zeros(len(image_cube))

for i, frame in enumerate(image_cube):
    bkg = Background2D(frame, (subarray_size, subarray_size), filter_size=(3, 3),
                       bkg_estimator=MedianBackground(), mask=source_mask)
    frame -= bkg.background
    phot_table = aperture_photometry(frame, aperture)
    light_curve[i] = phot_table['aperture_sum'][0]


In [ ]:
# Get times from filenames
start_times = [extract_time(file) for file in source_files_sorted]
# Convert start_times to seconds
start_times = np.array([int(time.split('_')[0]) * 3600 + int(time.split('_')[1]) * 60 + int(time.split('_')[2]) for time in start_times])
start_times = start_times.repeat(len(source_stack_0))
# Create numpy array with as many entries as image_cube, with time datatype
times = np.zeros(len(image_cube), dtype='datetime64[s]')
sub_start_times = np.linspace(0, 0.02 * len(source_stack_0), len(source_stack_0), endpoint=False)
# Duplicate sub_start_times len(start_times) times, appending itself to itself
sub_start_times = np.tile(sub_start_times, len(source_files_sorted))
frame_times = start_times + sub_start_times
# plt.plot(frame_times)
# plt.show()

In [ ]:
%matplotlib inline
plot_times = frame_times[:105]
plot_intensities = light_curve[:105]
plot_times -= np.min(plot_times)
# Normalize intensities
plot_intensities = plot_intensities / np.mean(plot_intensities)
# Smooth to 5 Hz
smoothed_times = np.zeros(21)
smoothed_intensities = np.zeros(21)
for i in range(21):
    smoothed_times[i] = np.mean(plot_times[i*5:(i+1)*5])
    smoothed_intensities[i] = np.mean(plot_intensities[i*5:(i+1)*5])
plt.xlabel('Time (s)', fontsize=16)
plt.ylabel('Normalized Intensity', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.scatter(plot_times, plot_intensities, s=2, c='blue', label='Data at 50 Hz')
plt.scatter(smoothed_times, smoothed_intensities, s=10, c='red', label='Data Smoothed to 5 Hz')
plt.legend()
plt.xlim(0, 2)
plt.ylim(0, 1.2)
plt.show()

In [ ]:
tno_intensities = [1.0000134698192191, 1.0000364392259375, 1.0000242504190036, 0.9999340083840196, 0.9998933258969123, 0.9999009353523838, 0.9999141473727281, 0.9999505354482887, 0.9999600023166005, 0.9998960228735443, 0.9998289730326697, 0.9997981021311697, 0.9998261833075585, 0.9998529667195817, 0.9998376775279177, 0.9998040404565443, 0.9998412111089414, 0.9998614759772817, 0.9999114404851637, 0.9999845788324442, 1.0000527647673763, 1.0000630744992192, 1.0000760265334445, 1.0002161018202749, 1.0002641165423383, 1.0002510882155866, 1.0008113185440495, 1.0011665929131166, 1.0000407226670276, 1.003604199195189, 1.0090715434469413, 1.0099983183468528, 0.9959830095039303, 0.9610020492180644, 0.9012434976073808, 0.8169728937408552, 0.7115435539519024, 0.6201346162771048, 0.5652046445981638, 0.5558984716534634, 0.592171960541998, 0.6649802964766569, 0.7629451159650922, 0.8563981055123481, 0.9260220372882716, 0.9763462921983986, 1.0037019357052879, 1.0110242600424613, 1.0082740144336921, 1.0020465102225753, 0.9997347024926875, 1.0014574207033666, 1.0007495558982782, 1.0000576802469427, 1.0003866287965488, 1.000210572104685, 0.9999661140372685, 0.9999515702151393, 0.999935230990795, 0.9998924370285919, 0.9998587897948424, 0.9998647061621334, 0.9998525287661911, 0.9998628135552516, 0.9998937626585946, 0.9999209283138302, 0.9999545899188365, 0.9999690295776567, 0.9999743811550289, 0.9999897793433837, 0.9999897518205481, 0.9999747220262055, 0.9999599259026961, 0.9999743755506212, 0.9999244957923341, 0.9999087854240389, 0.99986966341376, 0.9998466276833319, 0.9998462066680729]
print(len(tno_intensities))
# append 26 ones to tno_intensities
tno_intensities.extend([1] * 27)
injection_intensities = np.array(tno_intensities[:-1]) * plot_intensities
smoothed_injected_intensities = np.zeros(21)
for i in range(21):
    smoothed_injected_intensities[i] = np.mean(injection_intensities[i*5:(i+1)*5])
plt.xlabel('Time (s)', fontsize=16)
plt.ylabel('Normalized Intensity', fontsize=16)
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.scatter(plot_times, injection_intensities, s=2, c='blue', label='Data at 50 Hz, with injected occultation')
plt.scatter(smoothed_times, smoothed_injected_intensities, s=10, c='red', label='Data Smoothed to 50 Hz, with injected occultation')
plt.legend()
plt.xlim(0, 2)
plt.ylim(0, 1.2)
plt.show()

In [76]:
# EP241021 Analysis
# Star around 7130, 3094 is HD11730. EP241021 is near star at 2157, 2760
source_folder = os.path.join(data_folder, '11-01-2024/11 CMS EP241021A')
source_files = [file for file in os.listdir(source_folder) if file.endswith('.fits') and not file.startswith('.')]

# Function to extract time from filename
def extract_time(filename):
    match = re.search(r'(\d{2}_\d{2}_\d{2})\.fits$', filename)
    return match.group(1) if match else None

# Sort the files by extracted time
source_files_sorted = sorted(source_files, key=extract_time)
source_files_sorted = source_files_sorted[:]
print(source_files_sorted)
source_stack_0 = fits.open(os.path.join(source_folder, source_files_sorted[0]))[0].data
print(source_stack_0[0][2844, 2014])

dark_folder = os.path.join(data_folder, '11-01-2024/09 CMS Darks green')
dark_stack = fits.open(os.path.join(dark_folder, 'CMS_RS_dark_30000ms_-25C 2024 November 02 07_51_43.fits'))[0].data
print(np.mean(dark_stack))
flat_folder = os.path.join(data_folder, '11-01-2024/10 CMS Flats green')
flat_stack = fits.open(os.path.join(flat_folder, 'CMS_RS_dark_5000ms_-25C 2024 November 02 08_07_00.fits'))[0].data
dark_image = np.median(dark_stack, axis=0)
flat_image = np.median(flat_stack, axis=0)
# Normalize the flat image
flat_image /= np.median(flat_image)
plt.imshow(flat_image, vmin=0.9, vmax=1.1)
plt.colorbar()
plt.show()

['CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 06_34_50.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 06_47_25.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 08_26_21.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 08_28_47.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 08_41_22.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 08_53_57.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 09_06_32.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 09_19_07.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 09_31_42.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 09_44_17.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 09_56_52.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 10_09_27.fits', 'CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 10_22_01.fits']
1634
92.00140764608459


In [6]:
print(source_stack_0[0][2845,2015])

1707


In [7]:
%matplotlib qt
plt.imshow(source_stack_0[0], vmin=100, vmax=1000)
plt.show()

In [77]:
# Find the shifts that are necessary between frames to align them and outputs a final stacked iamge
%matplotlib inline
subarray_size = 100
image_cube = np.zeros((len(source_stack_0) * len(source_files_sorted), subarray_size, subarray_size))
# For 06_34_50
start_x, start_y = 2015, 2845
# For 08_26_21
# start_x, start_y = 2157, 2760
shift_x, shift_y = 0, 0
cum_shift_x, cum_shift_y = 0, 0
stacked_image = np.zeros((8120, 8120))

for i, file in enumerate(source_files_sorted[:]):
    print('Starting File %s' % file)
    if i == 2:
        start_x, start_y = 2157, 2760
        shift_x, shift_y = 0, 0
        cum_shift_x, cum_shift_y = 2015 - start_x, 2845 - start_y
    start_x -= shift_x
    start_y -= shift_y
    cum_shift_x += shift_x
    cum_shift_y += shift_y
    # Clear previous stack_data from memory
    stack_data = fits.open(os.path.join(source_folder, file))[0].data
    subarray_dark = dark_image[start_y - subarray_size // 2:start_y + subarray_size // 2,
                               start_x - subarray_size // 2:start_x + subarray_size // 2]
    subarray_flat = flat_image[start_y - subarray_size // 2:start_y + subarray_size // 2,
                               start_x - subarray_size // 2:start_x + subarray_size // 2]
    for j, frame in enumerate(stack_data):
        subarray_data = frame[start_y - subarray_size // 2:start_y + subarray_size // 2,
                              start_x - subarray_size // 2:start_x + subarray_size // 2]
        # plt.imshow(subarray_data)
        # plt.show()
        frame = (frame - dark_image) / flat_image
        subarray_data = (subarray_data - subarray_dark) / subarray_flat
        max_index = np.argmax(subarray_data)
        max_coords = np.unravel_index(max_index, subarray_data.shape)
        y, x = np.mgrid[:subarray_data.shape[0], :subarray_data.shape[1]]
        initial_guess = models.Gaussian2D(amplitude=subarray_data.max(), x_mean=max_coords[1], y_mean=max_coords[0], x_stddev=3, y_stddev=3)
        fitter = fitting.LevMarLSQFitter()
        fitted_gaussian = fitter(initial_guess, x, y, subarray_data)
        shift_x = np.rint(subarray_size / 2 - fitted_gaussian.x_mean.value).astype(int)
        shift_y = np.rint(subarray_size / 2 - fitted_gaussian.y_mean.value).astype(int)
        print(start_x, start_y, shift_x, shift_y)
        shifted_frame = shift_values(subarray_data, shift_x, shift_y)
        image_cube[i * len(source_stack_0) + j] = shifted_frame
        shifted_full_frame = shift_values(frame, cum_shift_x + shift_x, cum_shift_y + shift_y)
        stacked_image += shifted_full_frame
    del stack_data

Starting File CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 06_34_50.fits
2015 2845 0 0
2015 2845 0 1
2015 2845 0 0
2015 2845 -1 0
2015 2845 -2 0
2015 2845 -3 -1
2015 2845 -4 0
2015 2845 -4 0
2015 2845 -5 0
2015 2845 -5 0
2015 2845 -6 0
2015 2845 -7 -1
2015 2845 -8 0
2015 2845 -8 0
2015 2845 -9 0
2015 2845 -10 -1
2015 2845 -10 0
2015 2845 -11 0
2015 2845 -12 -1
2015 2845 -13 -2
2015 2845 -13 -1
2015 2845 -14 -1
2015 2845 -14 -1
2015 2845 -15 -1
2015 2845 -16 -1
Starting File CMS_RS_EP241021A_g_filter30000ms_-25C 2024-11-02 06_47_25.fits
2031 2846 0 -1
2031 2846 -1 0
2031 2846 -2 0
2031 2846 -3 -1
2031 2846 -3 0
2031 2846 -5 -1
2031 2846 -5 -1
2031 2846 -6 -1
2031 2846 -7 -1
2031 2846 -8 -1
2031 2846 -8 -1
2031 2846 -9 -1
2031 2846 -10 -2
2031 2846 -12 -2
2031 2846 -13 -2
2031 2846 -15 -2
2031 2846 -16 -2
2031 2846 -16 -2
2031 2846 -17 -2
2031 2846 -18 -2
2031 2846 -19 -2
2031 2846 -19 -2
2031 2846 -21 -2
2031 2846 -22 -3
2031 2846 -23 -3
Starting File CMS_RS_EP241021A_g_filter30000m

In [75]:
# Check that the shifting and adding worked with an animation
%matplotlib qt
fig, ax = plt.subplots()
im = ax.imshow(image_cube[0], cmap='gray')

def update(frame):
    im.set_array(image_cube[frame])
    return [im]

ani = animation.FuncAnimation(fig, update, frames=len(image_cube), interval=100, blit=True)
plt.show()

In [78]:
%matplotlib qt
bkg = Background2D(stacked_image, (8120, 8120), filter_size=(3, 3),
                    bkg_estimator=MedianBackground())
bkg_subtract_img = stacked_image - bkg.background
plt.imshow(bkg_subtract_img, vmin=0, vmax=10000)
plt.show()

: 

In [66]:
%matplotlib qt
# plt.imshow(stacked_image[3730:3830, 7300:7400])
# plt.show()
import copy
fit_star = copy.copy(stacked_image[3730:3830, 7300:7400].astype(float))
# plt.imshow(fit_star)
# plt.show()
max_index = np.argmax(fit_star)
positions = np.unravel_index(max_index, fit_star.shape)

subarray_size = 100
# Define the aperture
aperture = CircularAperture(positions, r=10)
# Create mask to calculate background
source_mask = aperture.to_mask(method='exact').to_image((subarray_size, subarray_size))
source_mask = source_mask.astype(bool)

bkg = Background2D(fit_star, (subarray_size, subarray_size), filter_size=(3, 3),
                    bkg_estimator=MedianBackground())
mean_bkg = np.mean(bkg.background)
fit_star -= bkg.background
# plt.imshow(fit_star)
# plt.show()
phot_table = aperture_photometry(fit_star, aperture)
reference_star_flux = phot_table['aperture_sum'][0]
print(reference_star_flux)
print(mean_bkg)
print(reference_star_flux / mean_bkg)


109547474.5239161
30787.75
3558.151359677667


In [ ]:
source_stack_0